In [ ]:
import pandas as pd
from time import time
!ls

import pyarrow.parquet as pq

pq_file = pq.ParquetFile("green_tripdata_2019-01.parquet")
print(pq_file.schema)

Dockerfile
Dockerfile.yaml
docker-compose.yaml
green_tripdata_2019-01.parquet
green_tripdata_2019-01.parquet:Zone.Identifier
ingesting.py
ingestring.py
main.ipynb
mainB.ipynb
notes,py
pgadmin_data
postgres_data
taxi_zone_lookup.csv
taxi_zone_lookup.csv:Zone.Identifier
required group field_id=-1 schema {
  optional int64 field_id=-1 VendorID;
  optional int64 field_id=-1 lpep_pickup_datetime (Timestamp(isAdjustedToUTC=false, timeUnit=microseconds, is_from_converted_type=false, force_set_converted_type=false));
  optional int64 field_id=-1 lpep_dropoff_datetime (Timestamp(isAdjustedToUTC=false, timeUnit=microseconds, is_from_converted_type=false, force_set_converted_type=false));
  optional binary field_id=-1 store_and_fwd_flag (String);
  optional double field_id=-1 RatecodeID;
  optional int64 field_id=-1 PULocationID;
  optional int64 field_id=-1 DOLocationID;
  optional double field_id=-1 passenger_count;
  optional double field_id=-1 trip_distance;
  optional double field_id=-1 fare

In [ ]:
import pyarrow.parquet as pq

table = pq.read_table("green_tripdata_2019-01.parquet")
green_trips_df = table.to_pandas()
green_trips_df.head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2018-12-21 15:17:29,2018-12-21 15:18:57,N,1.0,264,264,5.0,0.00,3.0,0.5,0.5,0.00,0.0,NaN,0.3,4.30,2.0,1.0,NaN
1,2,2019-01-01 00:10:16,2019-01-01 00:16:32,N,1.0,97,49,2.0,0.86,6.0,0.5,0.5,0.00,0.0,NaN,0.3,7.30,2.0,1.0,NaN
2,2,2019-01-01 00:27:11,2019-01-01 00:31:38,N,1.0,49,189,2.0,0.66,4.5,0.5,0.5,0.00,0.0,NaN,0.3,5.80,1.0,1.0,NaN
3,2,2019-01-01 00:46:20,2019-01-01 01:04:54,N,1.0,189,17,2.0,2.68,13.5,0.5,0.5,2.96,0.0,NaN,0.3,19.71,1.0,1.0,NaN
4,2,2019-01-01 00:19:06,2019-01-01 00:39:43,N,1.0,82,258,1.0,4.53,18.0,0.5,0.5,0.00,0.0,NaN,0.3,19.30,2.0,1.0,NaN
5,2,2019-01-01 00:12:35,2019-01-01 00:19:09,N,1.0,49,17,1.0,1.05,6.5,0.5,0.5,0.00,0.0,NaN,0.3,7.80,1.0,1.0,NaN
6,2,2019-01-01 00:47:55,2019-01-01 01:00:01,N,1.0,255,33,1.0,3.77,13.5,0.5,0.5,0.00,0.0,NaN,0.3,14.80,1.0,1.0,NaN
7,1,2019-01-01 00:12:47,2019-01-01 00:30:50,N,1.0,76,225,1.0,4.10,16.0,0.5,0.5,0.00,0.0,NaN,0.3,17.30,1.0,1.0,NaN
8,2,2019-01-01 00:16:23,2019-01-01 00:39:46,N,1.0,25,89,1.0,7.75,25.5,0.5,0.5,0.00,0.0,NaN,0.3,26.80,1.0,1.0,NaN
9,2,2019-01-01 00:58:02,2019-01-01 01:19:02,N,1.0,85,39,1.0,3.68,15.5,0.5,0.5,0.00,0.0,NaN,0.3,16.80,1.0,1.0,NaN


In [ ]:
zone_df = pd.read_csv("taxi_zone_lookup.csv")
zone_df.shape

(265, 4)

In [ ]:
from sqlalchemy import create_engine

PG_user = "lana"
PG_pass = "1234"
PG_host = "localhost"
PG_port = 5432
PG_db = "ny_taxi"

engine = create_engine(f"postgresql://{PG_user}:{PG_pass}@{PG_host}:{PG_port}/{PG_db}")
con = engine.connect()


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "lana"
connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "lana"

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
con

In [ ]:
# Generator function to read parquet in chunks
def read_parquet_in_chunks(file_path, chunksize=100000):
    parquet_file = pq.ParquetFile(file_path)
    for batch in parquet_file.iter_batches(batch_size=chunksize):
        yield batch.to_pandas()

# Iterator (mimics read_csv iterator)
df_iter = read_parquet_in_chunks("green_tripdata_2019-01.parquet", chunksize=10000)

# Load first chunk
df = next(df_iter)

# Convert datetime columns
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])

# Create table schema in SQL database (replace with your SQLAlchemy connection object)
df.head(0).to_sql(con=con, name="green_trip_data", if_exists="replace", index=False)



0

In [ ]:
df.to_sql(con=con, name="green_trip_data", if_exists="append", index=False)

1000

In [ ]:
try:
    while True:
        t_start = time()
        df = next(df_iter)

        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.to_sql(con=con, name="green_trip_data", if_exists="append", index=False)

        end_time = time()
        print("New records added {}".format(end_time - t_start))

except StopIteration:
    print("All record imported or ingested")



New records added 1.1591403484344482
New records added 1.1183700561523438
New records added 11.43429946899414
New records added 1.282663345336914
New records added 3.085742473602295
New records added 2.8520314693450928
New records added 1.2109386920928955
New records added 1.3190767765045166
New records added 1.0790727138519287
New records added 1.591601848602295
New records added 1.3134114742279053
New records added 1.795947790145874
New records added 1.6202561855316162
New records added 1.6560275554656982
New records added 3.036155939102173
New records added 2.4884603023529053
New records added 3.656827449798584
New records added 2.380063533782959
New records added 3.331960439682007
New records added 2.1522533893585205
New records added 3.4693872928619385
New records added 1.450113296508789
New records added 1.3147330284118652
New records added 3.227832078933716
New records added 1.1561403274536133
New records added 1.1378576755523682
New records added 1.346660852432251
New records a